In [ ]:
%pip install transformers torch pandas matplotlib sentence-transformers

In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

/Users/aceto/anaconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Prepare the data

In [2]:
SOURCE = "data/games.csv"

In [12]:
def process_games_data(source):
    # Read the CSV file
    games = pd.read_csv(source)

    # Select relevant columns
    games = games[['AppID', 'Name', 'About the game', 'Supported languages', 'Genres']]

    # Rename columns
    games.columns = ['id', 'name', 'description', 'languages', 'genres']

    # Drop rows with missing descriptions
    games = games.dropna(subset=['description'])

    # Filter games with English available
    games['english_available'] = games['languages'].apply(lambda x: 'english' in x.lower())
    games = games[games['english_available']]

    # Filter games with description length greater than 120
    games = games[games['description'].apply(lambda x: len(str(x))) > 120]

    # Truncate descriptions to 3000 characters
    games['description'] = games['description'].apply(lambda x: x[:3000])
    
    #Remove not needed columns
    games = games.drop(columns=['languages', 'english_available'])

    return games

In [13]:
model = SentenceTransformer("all-mpnet-base-v2")

/Users/aceto/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Experiment 1: Similarity only based on description

In [14]:
games_1 = process_games_data(SOURCE)
games_1['description'].describe()

count                                                 87945
unique                                                87463
top       Help the beautiful girls to immerse themselves...
freq                                                     65
Name: description, dtype: object

In [15]:
games_1

,id,name,description,genres
0,20200,Galactic Bowling,Galactic Bowling is an exaggerated and stylize...,"Casual,Indie,Sports"
1,655370,Train Bandit,THE LAW!! Looks to be a showdown atop a train....,"Action,Indie"
2,1732930,Jolt Project,Jolt Project: The army now has a new robotics ...,"Action,Adventure,Indie,Strategy"
3,1355720,Henosis™,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"Adventure,Casual,Indie"
4,1139950,Two Weeks in Painland,ABOUT THE GAME Play as a hacker who has arrang...,"Adventure,Indie"
...,...,...,...,...
97405,3080940,Femdom Game World: Mom,. Femdom Game World - is a fascinating series ...,Casual
97406,2593970,Blocky Farm,Enter the charming world of Blocky Farm where ...,"Casual,Simulation,Strategy"
97407,3137150,Infiltrate & Extract,Mission brief: You are deployed to a heavily g...,"Strategy,Free To Play"
97408,3124670,Escape The Garage,"Welcome to Escape The Garage, the thrilling es...","Adventure,Casual,Indie"


In [16]:
#games_1['embeddings'] = model.encode(games_1['description'].tolist())

KeyboardInterrupt: 

In [34]:
#get to similar games
g_1 = games_1[games_1["id"] == 849177]
g_2 = games_1[games_1["id"] == 391220]
#Get one not similar
g_3 = games_1[games_1["id"] == 1659180]
#Get embeddings
g_1['embeddings'] = [model.encode(g_1['description'].iloc[0]).tolist()]
g_2['embeddings'] = [model.encode(g_2['description'].iloc[0]).tolist()]
g_3['embeddings'] = [model.encode(g_3['description'].iloc[0]).tolist()]

/var/folders/3r/gx5x0f7n6lvd5yzgw2zddpv80000gn/T/ipykernel_28526/3141478878.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g_1['embeddings'] = [model.encode(g_1['description'].iloc[0]).tolist()]
/var/folders/3r/gx5x0f7n6lvd5yzgw2zddpv80000gn/T/ipykernel_28526/3141478878.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g_2['embeddings'] = [model.encode(g_2['description'].iloc[0]).tolist()]
/var/folders/3r/gx5x0f7n6lvd5yzgw2zddpv80000gn/T/ipykernel_28526/3141478878.py:9: SettingWithCopyWarning: 
A val

In [35]:
g_2['embeddings']

13400    [0.001636300585232675, 0.061147015541791916, -...
Name: embeddings, dtype: object

In [38]:
sim_1_2 = model.similarity(g_1['embeddings'].values[0], g_2['embeddings'].values[0])
sim_1_3 = model.similarity(g_1['embeddings'].values[0], g_3['embeddings'].values[0])
print(f"Similarity between {g_1['name'].values[0]} and {g_2['name'].values[0]}: {sim_1_2[0][0]} -> should be high")
print(f"Similarity between {g_1['name'].values[0]} and {g_3['name'].values[0]}: {sim_1_3[0][0]} -> should be low")

Similarity between Shadow of the Tomb Raider: Definitive Edition and Rise of the Tomb Raider™: 0.8550347089767456 -> should be high
Similarity between Shadow of the Tomb Raider: Definitive Edition and TD Worlds: 0.3918170928955078 -> should be low


## Experiment 2: Similarity based on description and genres (weighted approach)

In [41]:
games_2 = process_games_data(SOURCE)

Description embedding same as before

In [42]:
g_1 = games_2[games_2["id"] == 849177]
g_2 = games_2[games_2["id"] == 391220]
#Get one not similar
g_3 = games_2[games_2["id"] == 1659180]
#Get embeddings
g_1['desc_embeddings'] = [model.encode(g_1['description'].iloc[0]).tolist()]
g_2['desc_embeddings'] = [model.encode(g_2['description'].iloc[0]).tolist()]
g_3['desc_embeddings'] = [model.encode(g_3['description'].iloc[0]).tolist()]

/var/folders/3r/gx5x0f7n6lvd5yzgw2zddpv80000gn/T/ipykernel_28526/1140898956.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g_1['desc_embeddings'] = [model.encode(g_1['description'].iloc[0]).tolist()]
/var/folders/3r/gx5x0f7n6lvd5yzgw2zddpv80000gn/T/ipykernel_28526/1140898956.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g_2['desc_embeddings'] = [model.encode(g_2['description'].iloc[0]).tolist()]
/var/folders/3r/gx5x0f7n6lvd5yzgw2zddpv80000gn/T/ipykernel_28526/1140898956.py:8: SettingWithCopyWarni

In [71]:
g_1['desc_embeddings']

7776    [0.01750553771853447, 0.05562837794423103, -0....
Name: desc_embeddings, dtype: object

Genres embedding
- Option one would be one hot encoding and just see how many genres are in common
- Option two would be to use bert (single word) and then average the embeddings, get similarity

In [44]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
g_1['genres'] = g_1['genres'].apply(lambda x: x.split(', '))
g_2['genres'] = g_2['genres'].apply(lambda x: x.split(', '))
g_3['genres'] = g_3['genres'].apply(lambda x: x.split(', '))
#genres = mlb.fit_transform(games_2['genres'])
#genres = pd.DataFrame(genres, columns=mlb.classes_)
#genres

/var/folders/3r/gx5x0f7n6lvd5yzgw2zddpv80000gn/T/ipykernel_28526/1949706108.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g_1['genres'] = g_1['genres'].apply(lambda x: x.split(', '))
/var/folders/3r/gx5x0f7n6lvd5yzgw2zddpv80000gn/T/ipykernel_28526/1949706108.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g_2['genres'] = g_2['genres'].apply(lambda x: x.split(', '))
/var/folders/3r/gx5x0f7n6lvd5yzgw2zddpv80000gn/T/ipykernel_28526/1949706108.py:5: SettingWithCopyWarning: 
A value is trying to be set

TypeError: 'float' object is not iterable

In [49]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/Users/aceto/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [50]:
def get_genre_embeddings(genres: list):
    genre_embeddings = []
    for genre in genres:
        inputs = tokenizer(genre, return_tensors="pt")
        outputs = model(**inputs)
        genre_embeddings.append(outputs.last_hidden_state.mean(dim=1).detach().numpy())
    return np.mean(genre_embeddings, axis=0)

In [69]:
g_1['genre_embeddings'] = [get_genre_embeddings(g_1['genres'].iloc[0])]
g_2['genre_embeddings'] = [get_genre_embeddings(g_2['genres'].iloc[0])]
g_3['genre_embeddings'] = [get_genre_embeddings(g_3['genres'].iloc[0])]
#Flatten the saved lists
g_1['genre_embeddings'] = g_1['genre_embeddings'].apply(lambda x: x.tolist()[0])
g_2['genre_embeddings'] = g_2['genre_embeddings'].apply(lambda x: x.tolist()[0])
g_3['genre_embeddings'] = g_3['genre_embeddings'].apply(lambda x: x.tolist()[0])

/var/folders/3r/gx5x0f7n6lvd5yzgw2zddpv80000gn/T/ipykernel_28526/287222231.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g_1['genre_embeddings'] = [get_genre_embeddings(g_1['genres'].iloc[0])]
/var/folders/3r/gx5x0f7n6lvd5yzgw2zddpv80000gn/T/ipykernel_28526/287222231.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g_2['genre_embeddings'] = [get_genre_embeddings(g_2['genres'].iloc[0])]
/var/folders/3r/gx5x0f7n6lvd5yzgw2zddpv80000gn/T/ipykernel_28526/287222231.py:3: SettingWithCopyWarning: 
A value 

In [70]:
g_1['genre_embeddings'].iloc[0]

[-0.10142902284860611,
 -0.061262160539627075,
 0.09484749287366867,
 0.17833411693572998,
 0.3531375527381897,
 -0.5199637413024902,
 0.39155155420303345,
 0.20382559299468994,
 -0.2825397551059723,
 0.08408655971288681,
 -0.43010348081588745,
 0.22484596073627472,
 0.07554317265748978,
 0.5022488832473755,
 0.23771071434020996,
 0.2497600018978119,
 0.00610765814781189,
 0.0700867623090744,
 0.17763184010982513,
 0.1675715297460556,
 0.027022946625947952,
 -0.18263892829418182,
 0.0390826091170311,
 0.025111520662903786,
 0.4033457338809967,
 0.03307180106639862,
 -0.17355890572071075,
 -0.279629647731781,
 -0.28095346689224243,
 -0.13600359857082367,
 -0.1508440226316452,
 -0.15485021471977234,
 -0.32705071568489075,
 0.17028075456619263,
 0.07001380622386932,
 -0.23269128799438477,
 -0.22197452187538147,
 0.16080662608146667,
 -0.11361221969127655,
 0.029191672801971436,
 0.042666833847761154,
 -0.17990057170391083,
 0.31374722719192505,
 -0.1623702347278595,
 0.31818538904190063,


In [73]:
g_1

,id,name,description,genres,desc_embeddings,genre_embeddings
7776,849177,Shadow of the Tomb Raider: Definitive Edition,In Shadow of the Tomb Raider Definitive Editio...,"[Action,Adventure]","[0.01750553771853447, 0.05562837794423103, -0....","[-0.10142902284860611, -0.061262160539627075, ..."


In [78]:
#Function to calculate similarity based on desc_embeddings and genre_embeddings
from sklearn.metrics.pairwise import cosine_similarity
def weighted_similarity(game_1: list, game_2:list, desc_weight=0.8):
    similarity_desc = cosine_similarity(game_1[0], game_2[0])
    similarity_genre = cosine_similarity(game_1[1], game_2[1])
    return desc_weight * similarity_desc + (1 - desc_weight) * similarity_genre

In [79]:
sim_1_2 = weighted_similarity(g_1[['desc_embeddings', 'genre_embeddings']].values[0], g_2[['desc_embeddings', 'genre_embeddings']].values[0])

ValueError: Expected 2D array, got 1D array instead:
array=[ 1.75055377e-02  5.56283779e-02 -1.94122698e-02  2.07415745e-02
 -1.32416077e-02  5.34118153e-02 -6.43035769e-02  7.52757676e-03
 -5.63897975e-02  1.91594251e-02  2.75359172e-02  4.71493229e-02
  3.78632173e-03  9.03100893e-03  2.08571311e-02 -3.01131532e-02
  9.33512207e-03  1.23839052e-02 -1.05983384e-01  1.92305613e-02
 -3.98714468e-02 -2.23607779e-03 -1.06690507e-02 -3.29926908e-02
  6.78968951e-02  4.47034370e-04  1.02569526e-02  8.53046402e-02
 -2.42942758e-02 -4.22537103e-02  2.13581454e-02 -6.81235790e-02
  6.13739453e-02  1.03297585e-03  2.11824522e-06 -4.35234755e-02
 -3.03891841e-02 -2.69645955e-02 -5.83464727e-02  2.61926930e-03
 -4.89954427e-02  6.23812564e-02  6.49365364e-03 -5.38720079e-02
  4.18762155e-02  3.03339865e-03  5.21701723e-02  3.33647393e-02
  2.31434479e-02 -2.72726398e-02  2.69887364e-03 -4.55447845e-02
 -3.23101841e-02 -2.02460699e-02  6.93363994e-02 -9.54215357e-05
 -1.64552648e-02 -2.34200549e-0